In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
data = pd.read_csv(r"C:\Users\Admin\Data python\data_all\data_all\train.csv")
pd.set_option('display.max_columns', 32)
data.head()

,id,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,arrival_date
0,1,Resort Hotel,0,1,0,1,2,0.0,1,BB,PRT,Direct,Direct,0,0,0,A,A,1,250.0,0.0,0,Transient,55.00,1,0,2017-02-04
1,2,Resort Hotel,0,1,0,2,1,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,D,0,0.0,0.0,0,Transient,40.00,0,0,2016-03-04
2,3,City Hotel,1,56,0,3,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,29.0,0.0,0,Transient,60.00,0,0,2017-02-01
3,4,City Hotel,0,213,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,9.0,0.0,0,Transient,102.85,0,1,2016-10-05
4,5,Resort Hotel,0,328,4,10,2,0.0,0,BB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,243.0,0.0,0,Contract,121.50,0,2,2017-06-29


In [3]:
col = ['agent', 'company', 'id', "hotel"]

hotel_book = data.drop(col, axis= 1)

In [4]:
hotel_book.head()

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,arrival_date
0,0,1,0,1,2,0.0,1,BB,PRT,Direct,Direct,0,0,0,A,A,1,0,Transient,55.00,1,0,2017-02-04
1,0,1,0,2,1,0.0,0,BB,PRT,Direct,Direct,0,0,0,A,D,0,0,Transient,40.00,0,0,2016-03-04
2,1,56,0,3,2,0.0,0,BB,PRT,Groups,TA/TO,0,0,0,A,A,0,0,Transient,60.00,0,0,2017-02-01
3,0,213,2,5,2,0.0,0,BB,GBR,Online TA,TA/TO,0,0,0,A,A,0,0,Transient,102.85,0,1,2016-10-05
4,0,328,4,10,2,0.0,0,BB,GBR,Offline TA/TO,TA/TO,0,0,0,D,D,0,0,Contract,121.50,0,2,2017-06-29


In [5]:
hotel_book.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108785 entries, 0 to 108784
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   is_canceled                     108785 non-null  int64  
 1   lead_time                       108785 non-null  int64  
 2   stays_in_weekend_nights         108785 non-null  int64  
 3   stays_in_week_nights            108785 non-null  int64  
 4   adults                          108785 non-null  int64  
 5   children                        108785 non-null  float64
 6   babies                          108785 non-null  int64  
 7   meal                            108785 non-null  object 
 8   country                         108785 non-null  object 
 9   market_segment                  108785 non-null  object 
 10  distribution_channel            108785 non-null  object 
 11  is_repeated_guest               108785 non-null  int64  
 12  previous_cancell

In [6]:
hotel_book.describe()

,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,booking_changes,days_in_waiting_list,adr,required_car_parking_spaces,total_of_special_requests
count,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000,108785.000000
mean,0.370685,100.159847,0.909445,2.464935,1.845264,0.092375,0.007951,0.032541,0.094406,0.136958,0.211748,2.540249,97.085407,0.063097,0.544239
std,0.482991,104.953130,0.992503,1.887034,0.582494,0.377309,0.098157,0.177433,0.881879,1.448224,0.626862,18.401511,46.910659,0.246518,0.777150
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-6.380000,0.000000,0.000000
25%,0.000000,17.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,66.500000,0.000000,0.000000
50%,0.000000,65.000000,1.000000,2.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,90.000000,0.000000,0.000000
75%,1.000000,152.000000,2.000000,3.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,120.000000,0.000000,1.000000
max,1.000000,737.000000,19.000000,50.000000,55.000000,10.000000,10.000000,1.000000,26.000000,67.000000,18.000000,391.000000,5400.000000,8.000000,5.000000


## Explain column meaning

**hotel**：City hotel hoặc Resort Hotel

**lead_time**：Số ngày từ khi đặt phòng đến ngày nghỉ trọ.

**stays_in_weekend_nights**：Số ngày đặt phòng và nghỉ lại (Thứ 7, Chủ Nhật)

**stays_in_week_nights**：Số ngày đặt phòng và nghỉ lại (Thứ 2 ~ Thứ 6)

**country**：Quốc gia.

**market_segment**：Phân loại thị trường(TA：Đại lý du lịch、TO：Điều hành du lịch)

**distribution_channel**：kênh cung cấp(TA：Đại lý du lịch、TO：Điều hành du lịch)

**is_repeated_guest**：có phải repeater hay không

**previous_cancellations**：Đã từng hủy đặt phòng trước đây

**previous_bookings_not_canceled**：chưa từng hủy đặt phòng trước đây

**reserved_room_type**：Loại phòng đặt

**assigned_room_type**：Loại phòng sắp xếp

**booking_changes**：Số lần thay đổi đặt phòng

**agent**：ẩn

**company**：ẩn

**days_in_waiting_list**：Số ngày lấy được xác nhận của khách hàng.

**customer_type**：loại khách hàng

**adr**：Tỉ giá trung bình ngày nghỉ trọ

**required_car_parking_spaces**：Yêu cầu bao nhiêu chỗ đỗ xe

**total_of_special_requests**：Số lượng yêu cầu đặc biệt.

*Đây là bộ dữ liệu trích xuất từ hệ thống Quản lý khách sạn (PMS), vì thế tên khách sạn và đại lý đặt phòng sẽ được ẩn 
đi.*

### Explain for meal

**RO**: Room only

**BB**: Bed & Breakfast

**HB**: Half Board (Breakfast and Dinner normally)

**FB**: Full Board (Beakfast, Lunch and Dinner)

**AI**: All Inclusive (all services of full board plus any others specified in each case)

In [7]:
# Categorical data
col = ['meal', 'country',"market_segment", 'distribution_channel', 'reserved_room_type', 'assigned_room_type',"customer_type"]

for i in hotel_book[col]:
    x = hotel_book[i].value_counts()
    print("Column name is:",i,"and it value is:",x)
    print()

Column name is: meal and it value is: BB    85099
HB    12705
SC    10238
FB      743
Name: meal, dtype: int64

Column name is: country and it value is: PRT    45865
GBR    10764
FRA     9498
ESP     7614
DEU     6387
       ...  
FJI        1
MRT        1
CYM        1
SDN        1
UMI        1
Name: country, Length: 173, dtype: int64

Column name is: market_segment and it value is: Online TA        49545
Offline TA/TO    22682
Groups           19354
Direct           11196
Corporate         5079
Complementary      697
Aviation           230
Undefined            2
Name: market_segment, dtype: int64

Column name is: distribution_channel and it value is: TA/TO        88940
Direct       13203
Corporate     6449
GDS            188
Undefined        5
Name: distribution_channel, dtype: int64

Column name is: reserved_room_type and it value is: A    79728
D    16771
E     5708
F     2536
G     1796
B     1041
C      689
H      510
L        6
Name: reserved_room_type, dtype: int64

Column name 

**meal country**:5

**market_segment**: 177

**market_segment**: 8 

**distribution_channel**: 5 

**reserved_room_type**: 9

**assigned_room_type**: 11

**deposit_type**: 3

**customer_type**: 4

In [8]:
hotel_book.isnull().any()

is_canceled                       False
lead_time                         False
stays_in_weekend_nights           False
stays_in_week_nights              False
adults                            False
children                          False
babies                            False
meal                              False
country                           False
market_segment                    False
distribution_channel              False
is_repeated_guest                 False
previous_cancellations            False
previous_bookings_not_canceled    False
reserved_room_type                False
assigned_room_type                False
booking_changes                   False
days_in_waiting_list              False
customer_type                     False
adr                               False
required_car_parking_spaces       False
total_of_special_requests         False
arrival_date                      False
dtype: bool